本教程将帮助您熟悉LangChain的文档加载器、嵌入和向量存储抽象。这些抽象旨在支持从（向量）数据库和其他来源检索数据，以便与LLM工作流集成。它们对于在模型推理过程中获取数据进行推理的应用程序非常重要，例如检索增强生成（RAG）。

首先熟悉LangChain的文档读取操作。

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "materials/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [6]:
print(type(docs))
print(type(docs[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [4]:
print(f'{docs[0].page_content[:200]}\n')
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'materials/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


In [8]:
import chunk
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)
print(type(all_splits), len(all_splits))

<class 'list'> 516


接下来需要构建嵌入模型，嵌入模型是用来将文本嵌入为向量，并将语义映射为距离度量
可以使用api调用主流模型。这里方便起见使用ollama调用llama3模型。
```bash
pip install -qU langchain-ollama
```


In [10]:
from pyexpat import model
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model='llama3')

下面的代码将一个文本段映射为长度为4096的embedding

In [13]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 4096

[-0.0051661525, -0.029324213, -0.013686116, 0.004313129, -0.0062585142, -0.025107663, -0.0070748925, -0.0076910653, -0.034903385, -0.0007131876]


下面构建vector store，vector store可以看作一个能够通过embedding相似度查询的数据库。可以采用第三方提供的vector store，或者使用langchain提供的本地存储库
首先安装相关库
```bash
pip install -qU langchain-core
```

In [14]:
from langchain_core.vectorstores import InMemoryVectorStore

# Initialize the vector store with embeddings
vector_store = InMemoryVectorStore(embeddings)

In [16]:
ids = vector_store.add_documents(documents=all_splits)

In [18]:
len(ids)

516

In [19]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='Table of Contents
INTERNATIONAL MARKETS
For fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% and 61% for fiscal 2022 and fiscal 2021,
respectively. We sell our products to retail accounts through our own NIKE Direct operations and through a mix of independent distributors, licensees and sales
representatives around the world. We sell to thousands of retail accounts and ship products from 67 distribution centers outside of the United States. Refer to Item 2.
Properties for further information on distribution facilities outside of the United States. During fiscal 2023, NIKE's three largest customers outside of the United States
accounted for approximately 14% of total non-U.S. sales.
In addition to NIKE-owned and Converse-owned digital commerce platforms in over 40 countries, our NIKE Direct and Converse direct to consumer businesses operate
the following number of retail stores outside the United States:

In [21]:
len(results)

4

为了能够更高效地批量、异步查询，需要将VectorStore包装成Retrievers。

一种简单的方法是使用@chain注解，定义retriever函数中指定需要使用的查询方法和相关参数。

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

# 批量查询
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='1d1e9ea3-3e72-4c5d-83e2-be01145fbe7a', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'materials/nke-10k-2023.pdf', 'total_pages': 107, 'page': 5, 'page_label': '6', 'start_index': 0}, page_content="Table of Contents\nINTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% and 61% for fiscal 2022 and fiscal 2021,\nrespectively. We sell our products to retail accounts through our own NIKE Direct operations and through a mix of independent distributors, licensees and sales\nrepresentatives around the world. We sell 

另一种方法则是可以直接将VectorStore转化为Retriever。

In [22]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='1d1e9ea3-3e72-4c5d-83e2-be01145fbe7a', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'materials/nke-10k-2023.pdf', 'total_pages': 107, 'page': 5, 'page_label': '6', 'start_index': 0}, page_content="Table of Contents\nINTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% and 61% for fiscal 2022 and fiscal 2021,\nrespectively. We sell our products to retail accounts through our own NIKE Direct operations and through a mix of independent distributors, licensees and sales\nrepresentatives around the world. We sell 